#IMDB 영화평 감상분석 (이진분류)

In [2]:
import numpy as np
import pandas as pd
from google.colab import files
up = files.upload()

Saving labeledTrainData.tsv to labeledTrainData.tsv


In [3]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t')
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [7]:
df.review[100]

'"There is a uk edition to this show which is rather less extravagant than the US version. The person concerned will get a new kitchen or perhaps bedroom and bathroom and is wonderfully grateful for what they have got. The US version of this show is everything that reality TV shouldn\'t be. Instead of making a few improvements to a house which the occupants could not afford or do themselves the entire house gets rebuilt. I do not know if this show is trying to show what a lousy welfare system exists in the US or if you beg hard enough you will receive. The rather vulgar product placement that takes place, particularly by Sears, is also uncalled for. Rsther than turning one family in a deprived area into potential millionaires, it would be far better to help the community as a whole where instead of spending the hundreds of thousands of dollars on one home, build something for the whole community ..... perhaps a place where diy and power tools can be borrowed and returned along with bui

In [10]:
# 텍스트 전처리
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />',' ')

# 구둣점, 숫제는 제거, 영문자가 아닌 글자는 공백으로 변환 (re를 쓰지 않고도 데이터프레임의 str method는 정규표현식을 지원함)
df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()
df.review[100]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


'There is a uk edition to this show which is rather less extravagant than the US version  The person concerned will get a new kitchen or perhaps bedroom and bathroom and is wonderfully grateful for what they have got  The US version of this show is everything that reality TV shouldn t be  Instead of making a few improvements to a house which the occupants could not afford or do themselves the entire house gets rebuilt  I do not know if this show is trying to show what a lousy welfare system exists in the US or if you beg hard enough you will receive  The rather vulgar product placement that takes place  particularly by Sears  is also uncalled for  Rsther than turning one family in a deprived area into potential millionaires  it would be far better to help the community as a whole where instead of spending the hundreds of thousands of dollars on one home  build something for the whole community       perhaps a place where diy and power tools can be borrowed and returned along with build

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( df.review, df.sentiment, stratify=df.sentiment, random_state=2022)
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cvect =CountVectorizer(stop_words = 'english')


In [15]:
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_train_cv.shape

(18750, 65213)

In [16]:
# test dataset은 train dataset을 변환한 Vectorizer로 변환해야함. 

X_test_cv = cvect.transform(X_test)
X_test_cv.shape

(6250, 65213)

In [20]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()


In [22]:
%time nb.fit(X_train_cv, y_train)

CPU times: user 41.9 ms, sys: 997 µs, total: 42.9 ms
Wall time: 48.7 ms


MultinomialNB()

In [23]:
nb.score(X_test_cv, y_test)

0.85504

In [24]:
cvect2 = CountVectorizer(ngram_range=(1,2), stop_words='english')
cvect2.fit(X_train)
%time X_train_cv2 = cvect2.transform(X_train)
print(X_train_cv2.shape)

CPU times: user 5.88 s, sys: 18 ms, total: 5.9 s
Wall time: 5.91 s
(18750, 1386558)


In [25]:
X_test_cv2 = cvect2.transform(X_test)
X_test_cv2.shape

(6250, 1386558)

In [26]:
nb2 = MultinomialNB()
nb2.fit(X_train_cv2, y_train)

MultinomialNB()

In [27]:
nb2.score(X_test_cv2, y_test)

0.8672

- 모델 저장하고 불러오기

In [28]:
import joblib

In [29]:
joblib.dump(cvect2, 'imdb_cvect12.pkl')
joblib.dump(nb2, 'imdb_nb2.pkl')

['imdb_nb2.pkl']

In [30]:
!ls -l

total 114376
-rw-r--r-- 1 root root 39184702 Apr 28 02:39 imdb_cvect12.pkl
-rw-r--r-- 1 root root 44370573 Apr 28 02:39 imdb_nb2.pkl
-rw-r--r-- 1 root root 33556378 Apr 28 01:36 labeledTrainData.tsv
drwxr-xr-x 1 root root     4096 Apr 25 13:46 sample_data


In [31]:
new_cvect = joblib.load('imdb_cvect12.pkl')
new_nb = joblib.load('imdb_nb2.pkl')


In [32]:
#실제 데이터로 검증
review = '''

Robert P is a terrible Batman. Absolutely no Bat-charisma at all. The Riddler character is a complete bore! I hated all of this film. I certainly do not understand the good reviews. It's not close to the Dark Knight caliber. Please do not make a sequel.
'''


In [33]:
#텍스트 전처리
import re
review = re.sub('[^A-Za-z]', ' ', review).strip()


In [34]:
#feature 변환
review_cv = new_cvect.transform([review])
review_cv.shape


(1, 1386558)

In [35]:
new_nb.predict(review_cv)

array([0])